In [2]:
#! /usr/bin/env python3

from pymarc import MARCReader
import os
import csv

In [3]:
with open('source_data/MARCDATA.MRC', 'rb') as f:
    reader = MARCReader(f)
    all_records = list()
    for record in reader:
        all_records.append(record)

In [4]:
# an example of one record
all_records[1000].as_dict()

{'fields': [{'001': 'AAI3167161'},
  {'005': '20140908111943.5'},
  {'008': '140908s2005    ||||||||||||||||| ||eng d'},
  {'020': {'ind1': ' ', 'ind2': ' ', 'subfields': [{'a': '9780542025891'}]}},
  {'035': {'ind1': ' ',
    'ind2': ' ',
    'subfields': [{'a': '(MiAaPQ)AAI3167161'}]}},
  {'040': {'ind1': ' ',
    'ind2': ' ',
    'subfields': [{'a': 'MiAaPQ'}, {'c': 'MiAaPQ'}]}},
  {'100': {'ind1': '1', 'ind2': ' ', 'subfields': [{'a': 'Song, In-hyouk.'}]}},
  {'245': {'ind1': '1',
    'ind2': '0',
    'subfields': [{'a': 'Laterally movable gate field effect transistor (LMGFET) for microsensor and microactuator applications.'}]}},
  {'300': {'ind1': ' ', 'ind2': ' ', 'subfields': [{'a': '171 p.'}]}},
  {'500': {'ind1': ' ',
    'ind2': ' ',
    'subfields': [{'a': 'Source: Dissertation Abstracts International, Volume: 66-03, Section: B, page: 1638.'}]}},
  {'500': {'ind1': ' ',
    'ind2': ' ',
    'subfields': [{'a': 'Director: Pratul K. Ajmera.'}]}},
  {'502': {'ind1': ' ',
    'i

In [5]:
def lookup_uid(record):
    return record.get_fields('001')[0].value().replace('AAI','')

In [6]:
# lookup_uid(all_records[0])

In [7]:
# make a list of restricted items
# from the previously-made ranges
# they match 1247 records from the marc file

restricted_range_a = set(range(3048322, 3335145))
restricted_range_b = {3021429, 3030348, 3451495}
restricted_range_c = {3049191, 3049223, 3051440, 3053695, 3053696}
restricted_range_d = set(range(3049188, 3329096))
all_restricteds = set().union(restricted_range_a, 
                              restricted_range_b,
                              restricted_range_c,
                              restricted_range_d)
all_restricteds.remove(3136164)

restricted_uids = []

for record in all_records:
    uid = lookup_uid(record)
    if int(uid) in all_restricteds:
        restricted_uids.append(uid)
        continue
        
# print(len(restricted_uids))

In [8]:
# make a list of duplicated uids

with open('source_data/DuplicatedInDigitalCommons.txt', 'r', encoding='utf-8') as f:
    duplicate_uids = []
    for line in f.readlines():
        duplicate_uids.append(line.replace('.pdf', '').strip())

In [9]:
# make a list of records not in duplicated or in restricted

to_do_records = [i for i in all_records if lookup_uid(i) not in restricted_uids
                                        and lookup_uid(i) not in duplicate_uids]


## Testing things

In [10]:
# test of all uids in marc file match a pdf on U drive
# short answer: they all do

# pdf_not_on_U = list()

# for record in all_records:
#     uid = lookup_uid(record)
#     if os.path.isfile('/media/francis/U/ProquestDissertations/UnrestrictedTheses/{}.pdf'.format(uid)):
#         continue
#     pdf_not_on_U.append(uid)

# print(pdf_not_on_U)


In [11]:
# show all unique values for field 650

# all_650a = set()
# for record in to_do_records:
#     for i in record.get_fields('650'):
#         all_650a.add(i.value())
#     all_650a.add(record.get_fields('650')[0].value())
# print(all_650a)

## Making the crosswalk

## Making the csv

In [12]:
def csv_writer(data, path):
    with open(path, "w", newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for line in data:
            writer.writerow(line)

In [13]:
def build_csv(all_data_parsed):
    csv_data = []
    # first row is all fieldnames, following rows are each uid's info
    
    csvfieldnames = ["title",
                     "fulltext_url",
                     'keywords',
                     'abstract',
                     "author1_fname",
                     'author1_mname',
                     'author1_lname',
                     'author1_suffix',
                     'author1_email',
                     'author1_institution',
                     'advisor1',
                     'advisor2',
                     'advisor3',
                     'disciplines',
                     'comments'
                     'degree_name',
                     'department',
                     "document_type",
                     'publication_date',
                     'season',
                     'release_date',
                     'urn',
                    ]
    csv_data.append(csvfieldnames)

    for record in to_do_uids:
        csv_title = record.get_fields('245')
        fulltext_url = parse_dropbox_url(urn)
#             fulltext_url = gdrive_dict[urn]
         csv_keywords =
         csv_abstract =
         csv_first_name =
         csv_middle_name =
         csv_last_name =
         csv_suffix =
         csv_author_email =
         csv_institution =
         csv_advisor1 =
         csv_advisor2 =
         csv_advisor3 =
         csv_disciplines =
         csv_comments =
         csv_degree_name =
         csv_department =
         csv_document_type =
         csv_publication_date =
         csv_season =
         csv_release_date =
         csv_urn =



        csv_data.append([csv_title,
                         fulltext_url,
                         csv_keywords,
                         csv_abstract,
                         csv_first_name,
                         csv_middle_name,
                         csv_last_name,
                         csv_suffix,
                         csv_author_email,
                         csv_institution,
                         csv_advisor1,
                         csv_advisor2,
                         csv_advisor3,
                         csv_disciplines,
                         csv_comments,
                         csv_degree_name,
                         csv_department,
                         csv_document_type,
                         csv_publication_date,
                         csv_season,
                         csv_release_date,
                         csv_urn,
                         ])
#     print(csv_data)
    csv_writer(csv_data, 'output/scrap_Proquest.csv')

IndentationError: unexpected indent (<ipython-input-13-e4853881ec61>, line 34)

In [14]:
# how many unique values for each field/subfield?

# counting_items = dict()

# def add_to_if_not_yet(k, v):
#     v = v.strip()
#     if v == "None" or not v or v == None:
#         return
#     if k in counting_items:
#         counting_items[k].add(v)
#     else:
#         counting_items[k] = set()
#         counting_items[k].add(v)

# for record_as_marc in to_do_records:
#     record = record_as_marc.as_dict()
#     if not record['fields']:
#         break
#     for dictionary in record['fields']:
#         for k, v in dictionary.items():
#             if isinstance(v, str) and v:
#                 add_to_if_not_yet(k, v)
#             if isinstance(v, dict) and v:
#                 ind1 = v['ind1']
#                 fullpath = '{}/ind1'.format(k)
#                 add_to_if_not_yet(fullpath, ind1)
#                 ind2 = v['ind2']
#                 fullpath = '{}/ind2'.format(k)
#                 add_to_if_not_yet(fullpath, ind2)
#                 subfields = v['subfields']
#                 for subdictionary in subfields:
#                     for x, y in subdictionary.items():
#                         fullpath = '{}/subfields/{}'.format(k, x)
#                         add_to_if_not_yet(fullpath, y)
                        
# for k, v in counting_items.items():
#     print(k, len(v))

In [15]:
# how many unique values for each field/subfield?

# keys_lengths = dict()

# def add_to_if_not_yet(k, v):
#     if k in keys_lengths:
#         keys_lengths[k].add(v)
#     else:
#         keys_lengths[k] = set()
#         keys_lengths[k].add(v)

# for record_as_marc in to_do_records:
#     record = record_as_marc.as_dict()
#     if not record['fields']:
#         break        
#     field_keys = {k for field in record['fields'] for k in field.keys()}
#     fields_list = [k for field in record['fields'] for k in field.keys()]
#     for unique_field in field_keys:
#         add_to_if_not_yet(unique_field, fields_list.count(unique_field))

# print(keys_lengths)

In [27]:
# this is supposed to check for broken utf-8, but i don't trust it's working

# longest_field = 0

# for record_as_marc in to_do_records:
#     for field in record_as_marc.get_fields():
#         value = field.value()
#         try:
#             bytes_value = value.encode()
#             ascii_value = bytes_value.decode('ascii', "strict")
#             if len(ascii_value) > longest_field:
#                 longest_field = len(ascii_value)
#                 print(record_as_marc)
#         except:
#             print(value)


=LDR  00974nam a2200265   4500
=001  AAI0000418
=005  20140908111845.5
=008  140908s1941\\\\|||||||||||||||||\||eng\d
=035  \\$a(MiAaPQ)AAI0000418
=040  \\$aMiAaPQ$cMiAaPQ
=100  1\$aFLOWERS, FRANK C.
=245  10$aMARK TWAIN'S THEORIES OF MORALITY.
=300  \\$a239 p.
=500  \\$aSource: Dissertation Abstracts International, Volume: 04-01, page: 2700.
=502  \\$aThesis (Ph.D.)--Louisiana State University and Agricultural & Mechanical College, 1941.
=590  \\$aSchool code: 0107.
=650  \4$aPhilosophy.
=690  \\$a0422
=710  20$aLouisiana State University and Agricultural & Mechanical College.
=773  0\$tDissertation Abstracts International$g04-01.
=790  \\$a0107
=791  \\$aPh.D.
=792  \\$a1941
=793  \\$aEnglish
=856  \\$uhttp://gateway.proquest.com/openurl?url_ver=Z39.88-2004&rft_val_fmt=info:ofi/fmt:kev:mtx:dissertation&res_dat=xri:pqm&rft_dat=xri:pqdiss:0000418

=LDR  00974nam a2200265   4500
=001  AAI0000418
=005  20140908111845.5
=008  140908s1941\\\\|||||||||||||||||\||eng\d
=035  \\$a(MiAaPQ)AAI0